# Case Study 1 - Data Modality

## The Task
Get used to loading datasets with the library and generating synthetic data from them, whatever the modality of the real data.

### Imports
Lets get the imports out of the way. We import the required standard and 3rd party libraries and relevant Synthcity modules. We can also set the level of logging here, using Synthcity's bespoke logger. 

In [ ]:
# Standard
import sys
import warnings
from pathlib import Path

# 3rd party
import numpy as np
import pandas as pd

# synthcity
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import (GenericDataLoader, SurvivalAnalysisDataLoader, TimeSeriesDataLoader,TimeSeriesSurvivalDataLoader)

# Configure warnings and logging
warnings.filterwarnings("ignore")

# Set the level for the logging
# log.add(sink=sys.stderr, level="DEBUG")
log.remove()

## Synthetic generators

We can list the available generic synthetic generators by calling list() on the Plugins object.

In [ ]:
print(Plugins().list())

## Loading data of different modalities
### Static Data
Lets start with the simplest example static tabular data. First we need to load the dataset.

In [ ]:
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True, as_frame=True)
X["target"] = y
display(X)


Then we need to pass it to the `GenericDataLoader` object.

In [ ]:
loader = GenericDataLoader(
    X,
    target_column="target",
    sensitive_columns=["sex"],
)

We can print out different methods that are compatible with our data by calling `Plugins().list()` with a relevant list passed to the categories parameter.

In [ ]:
print(Plugins(categories=["generic"]).list())

No need to worry about the code in this next block here, we will go into lots of detail in how to generate synthetic data in the case studies to come. It is here purely to demonstrate that our dataset can be used to generate synthetic data using the synthcity module.

In [ ]:
syn_model = Plugins().get("marginal_distributions")
syn_model.fit(loader)
syn_model.generate(count=10).dataframe()

## Static survival
Show censoring without time series. We allow composite data, i.e. mixed static and temporal datasets.

### Regular Time Series

Now lets load a time series dataset and show that that is also compatible with Synthcity.

In [ ]:
import numpy as np
from synthcity.utils.datasets.time_series.google_stocks import GoogleStocksDataloader

static_data, temporal_data, horizons, outcome = GoogleStocksDataloader().load()
loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,
    observation_times=horizons,
    static_data=static_data,
    outcome=outcome,
)
print(loader.info())

syn_model = Plugins().get("marginal_distributions", n_iter=1)
syn_model.fit(loader)
syn_model.generate(count=10)
display(syn_model.generate(count=10).dataframe())

### Irregular Time Series 

### Composite Irregular Time Series Survival Analysis

Lets add it all together!

This next dataset is a composite irregular time series survival analysis dataset. Even complex datasets such as this are compatible with Synthcity. By calling `loader.info()`, we can see information about the dataset. We can see that it contains both one static feature and 14 temporal features, making it a composite dataset. The `seq_time_id` field shows the irregular time sampling, which we create by passing the values to the `observation_times` parameter of the `TimeSeriesSurvivalDataLoader` object. And finally, we are formulating this data as a survival analysis problem, which is indicated by the presence of a `time_to_event` field.

In [ ]:
import numpy as np
from synthcity.utils.datasets.time_series.pbc import PBCDataloader

(
    static_surv,
    temporal_surv,
    temporal_surv_horizons,
    outcome_surv,
) = PBCDataloader().load()
T, E = outcome_surv

horizons = [0.25, 0.5, 0.75]
time_horizons = np.quantile(T, horizons).tolist()

loader = TimeSeriesSurvivalDataLoader(
    temporal_data=temporal_surv,
    observation_times=temporal_surv_horizons,
    static_data=static_surv,
    T=T,
    E=E,
    time_horizons=time_horizons,
)

print(loader.info())

syn_model = Plugins().get("marginal_distributions", n_iter=1)
syn_model.fit(loader)
syn_model.generate(count=10)
display(syn_model.generate(count=10).dataframe())

### Create synthetic datasets
 - Above we have generated data with the debugging method `"marginal_distributions"`. Now, using `Plugins().list()` or the documentation find another method that is compatible with some of the datasets to see if you can generate your own snthetic data.